In [1]:
print("ok")

ok


 ##### This notebook is for testing core components of our AI voice agent for restaurants. We'll test:,
    "1. Twilio API integration\n",
    "2. Speech recognition and intent classification\n",
    "3. Database operations\n",
    "4. Context awareness\n",
    "5. End-to-end call flow"

In [1]:
#phase 1

In [2]:
import os
import json
import datetime
from dotenv import load_dotenv
import openai
from twilio.rest import Client
from twilio.twiml.voice_response import VoiceResponse, Gather
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Boolean, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Load environment variables
load_dotenv()

# Configure OpenAI
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configure Twilio
twilio_account_sid = os.getenv('TWILIO_ACCOUNT_SID')
twilio_auth_token = os.getenv('TWILIO_AUTH_TOKEN')
twilio_phone_number = os.getenv('TWILIO_PHONE_NUMBER')

# Initialize Twilio client
twilio_client = Client(twilio_account_sid, twilio_auth_token)

In [3]:
# Set up SQLite for testing
Base = declarative_base()

class Order(Base):
    __tablename__ = 'orders'
    
    id = Column(Integer, primary_key=True)
    customer_name = Column(String(100), nullable=False)
    customer_phone = Column(String(20), nullable=False)
    order_items = Column(Text, nullable=False)  # JSON string of order items
    order_total = Column(Integer)  # Total in cents
    reservation_time = Column(DateTime, nullable=True)
    party_size = Column(Integer, nullable=True)
    status = Column(String(20), default='confirmed')  # confirmed, modified, cancelled
    created_at = Column(DateTime, default=datetime.datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.datetime.utcnow, onupdate=datetime.datetime.utcnow)
    
class Conversation(Base):
    __tablename__ = 'conversations'
    
    id = Column(Integer, primary_key=True)
    call_sid = Column(String(50), nullable=False)
    customer_phone = Column(String(20), nullable=False)
    conversation_log = Column(Text, nullable=False)  # JSON string of conversation
    order_id = Column(Integer, nullable=True)  # If an order was created/modified
    created_at = Column(DateTime, default=datetime.datetime.utcnow)

# Create database engine and tables
engine = create_engine('sqlite:///restaurant_bot_test.db')
Base.metadata.create_all(engine)

# Create session factory
Session = sessionmaker(bind=engine)
session = Session()

C:\Users\Waleed\AppData\Local\Temp\ipykernel_14436\1205694561.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [4]:
def classify_intent(transcript):
    """Classify the intent of user's speech using OpenAI"""
    
    system_prompt = """
    You are an AI assistant for a restaurant. Classify the customer's intent into one of the following categories:
    - new_order: Customer wants to place a new order
    - modify_order: Customer wants to modify an existing order
    - cancel_order: Customer wants to cancel an order
    - check_status: Customer wants to check order status
    - general_inquiry: Customer has a general question
    - end_call: Customer wants to end the call
    - unclear: Intent is not clear
    
    Return only the category as a single word.
    """
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",  # Using a smaller, faster model for intent classification
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": transcript}
        ],
        max_tokens=10,
        temperature=0.3
    )
    
    intent = response.choices[0].message.content.strip().lower()
    return intent

# Test intent classification
test_statements = [
    "I'd like to order a pizza for delivery",
    "Can I change my order? I'd like to add a salad",
    "I need to cancel my order",
    "What time will my food arrive?",
    "Do you have gluten-free options?",
    "That's all, thank you. Goodbye."
]

for statement in test_statements:
    intent = classify_intent(statement)
    print(f"Statement: '{statement}'\nIntent: {intent}\n")

Statement: 'I'd like to order a pizza for delivery'
Intent: new_order

Statement: 'Can I change my order? I'd like to add a salad'
Intent: modify_order

Statement: 'I need to cancel my order'
Intent: cancel_order

Statement: 'What time will my food arrive?'
Intent: check_status

Statement: 'Do you have gluten-free options?'
Intent: general_inquiry

Statement: 'That's all, thank you. Goodbye.'
Intent: end_call



In [5]:
def generate_response(transcript, conversation_history, order_data=None):
    """Generate AI response based on customer's transcript and conversation history"""
    
    system_prompt = """
    You are an AI assistant for Mario's Italian Restaurant. Your name is Mario's Virtual Assistant.
    You handle phone orders and reservations politely and efficiently.
    
    Restaurant details:
    - Hours: Tuesday-Sunday, 11am-10pm (closed Mondays)
    - Specials: Margherita Pizza ($16), Seafood Linguine ($22), Tiramisu ($8)
    - Delivery available within 5 miles, $3 delivery fee
    - Reservations needed for parties of 5 or more
    
    When taking orders or making reservations:
    1. Get customer name
    2. Get order details or reservation time/party size
    3. Confirm details before finalizing
    4. End with a polite message
    
    Keep responses conversational but concise (max 3 sentences).
    If you can't help or understand, politely offer to transfer to a human.
    """
    
    # Prepare messages including conversation history
    messages = [
        {"role": "system", "content": system_prompt}
    ]
    
    # Add conversation history
    for exchange in conversation_history:
        messages.append({"role": "user", "content": exchange["customer"]})
        if "assistant" in exchange:
            messages.append({"role": "assistant", "content": exchange["assistant"]})
    
    # Add current transcript
    messages.append({"role": "user", "content": transcript})
    
    # Add order data if available
    if order_data:
        order_context = f"Customer has an existing order: {json.dumps(order_data)}"
        messages.append({"role": "system", "content": order_context})
    
    response = openai.chat.completions.create(
        model="gpt-4o",  # Using more capable model for complete responses
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )
    
    return response.choices[0].message.content

# Test response generation
conversation_history = [
    {"customer": "Hi, I'd like to order some food for delivery."}
]

response = generate_response("Do you have any vegetarian options?", conversation_history)
conversation_history.append({"customer": "Do you have any vegetarian options?", "assistant": response})
print(f"AI: {response}\n")

response = generate_response("I'll take a Margherita Pizza and Tiramisu.", conversation_history)
print(f"AI: {response}")

AI: Yes, we do! Our Margherita Pizza is a great vegetarian option. Would you like to place an order for that, or is there something else you'd prefer?

AI: Great choice! May I have your name and delivery address, please? Also, could you confirm if you're within our 5-mile delivery range?


In [6]:
def generate_twiml_response(ai_message, gather_speech=True):
    """Generate TwiML for voice response"""
    response = VoiceResponse()
    
    # Add the AI's message
    response.say(ai_message, voice='Polly.Joanna')
    
    # If we want to gather speech response
    if gather_speech:
        gather = Gather(
            input='speech',
            action='/voice/process-speech',
            timeout=3,
            speech_timeout='auto',
            language='en-US'
        )
        response.append(gather)
        
        # If no input is received, retry
        response.redirect('/voice/handle-no-input')
    
    return response

# Test TwiML generation
test_message = "Thank you for calling Mario's Italian Restaurant. How can I assist you today?"
twiml_response = generate_twiml_response(test_message)
print(twiml_response)

<?xml version="1.0" encoding="UTF-8"?><Response><Say voice="Polly.Joanna">Thank you for calling Mario's Italian Restaurant. How can I assist you today?</Say><Gather action="/voice/process-speech" input="speech" language="en-US" speechTimeout="auto" timeout="3" /><Redirect>/voice/handle-no-input</Redirect></Response>


In [7]:
def parse_order_details(transcript, conversation_history):
    """Extract order details from conversation"""
    
    system_prompt = """
    Extract order details from the conversation. Return a JSON object with these fields:
    - customer_name: customer's name or null if unknown
    - order_items: array of items ordered with quantity and special instructions
    - is_delivery: boolean indicating if delivery is requested
    - address: delivery address or null if not provided
    - reservation_time: datetime string if a reservation was made or null
    - party_size: number of people if a reservation was made or null
    
    Only include information explicitly stated in the conversation.
    """
    
    # Prepare the full conversation
    full_conversation = ""
    for exchange in conversation_history:
        if "customer" in exchange:
            full_conversation += f"Customer: {exchange['customer']}\n"
        if "assistant" in exchange:
            full_conversation += f"Assistant: {exchange['assistant']}\n"
    
    full_conversation += f"Customer: {transcript}"
    
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": full_conversation}
        ],
        response_format={"type": "json_object"},
        max_tokens=500,
        temperature=0.2
    )
    
    try:
        order_details = json.loads(response.choices[0].message.content)
        return order_details
    except json.JSONDecodeError:
        # Fallback for parsing errors
        return {
            "customer_name": None,
            "order_items": [],
            "is_delivery": False,
            "address": None,
            "reservation_time": None,
            "party_size": None,
            "parsing_error": True
        }

def save_order(order_details, customer_phone):
    """Save order to database"""
    new_order = Order(
        customer_name=order_details.get("customer_name", "Unknown"),
        customer_phone=customer_phone,
        order_items=json.dumps(order_details.get("order_items", [])),
        order_total=calculate_order_total(order_details.get("order_items", [])),
        reservation_time=parse_datetime(order_details.get("reservation_time")),
        party_size=order_details.get("party_size")
    )
    
    session.add(new_order)
    session.commit()
    return new_order.id

def calculate_order_total(order_items):
    """Calculate total order cost in cents - simplified for testing"""
    # Mock menu prices
    menu_prices = {
        "margherita pizza": 1600,
        "seafood linguine": 2200,
        "tiramisu": 800,
        # Add more menu items as needed
    }
    
    total = 0
    for item in order_items:
        item_name = item.get("item", "").lower()
        quantity = item.get("quantity", 1)
        
        # Try to find the price, defaulting to 1000 cents if unknown
        price = menu_prices.get(item_name, 1000)
        total += price * quantity
    
    return total

def parse_datetime(datetime_str):
    """Parse datetime string to datetime object"""
    if not datetime_str:
        return None
    
    try:
        return datetime.datetime.fromisoformat(datetime_str)
    except (ValueError, TypeError):
        try:
            # Try another common format
            return datetime.datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
        except (ValueError, TypeError):
            return None

In [9]:
def simulate_call():
    """Simulate a complete call flow"""
    print("=== Simulating Call Flow ===")
    
    # Initialize call
    customer_phone = "+15551234567"
    call_sid = "SIM12345678"
    conversation_history = []
    
    # Initial greeting
    ai_greeting = "Thank you for calling Mario's Italian Restaurant. This is Mario's virtual assistant. How can I help you today?"
    print(f"Assistant: {ai_greeting}")
    
    # Customer response 1
    customer_response_1 = "Hi, I'd like to place an order for delivery."
    print(f"Customer: {customer_response_1}")
    
    # Process intent
    intent = classify_intent(customer_response_1)
    print(f"Intent classified as: {intent}")
    
    # Update conversation history
    conversation_history.append({"customer": customer_response_1})
    
    # Generate response
    ai_response_1 = generate_response(customer_response_1, conversation_history)
    print(f"Assistant: {ai_response_1}")
    
    # Update conversation history
    conversation_history[-1]["assistant"] = ai_response_1
    
    # Customer response 2
    customer_response_2 = "I'd like a Margherita pizza and a tiramisu for dessert. My name is John."
    print(f"Customer: {customer_response_2}")
    
    # Update conversation history
    conversation_history.append({"customer": customer_response_2})
    
    # Generate response
    ai_response_2 = generate_response(customer_response_2, conversation_history)
    print(f"Assistant: {ai_response_2}")
    
    # Update conversation history
    conversation_history[-1]["assistant"] = ai_response_2
    
    # Parse order details
    order_details = parse_order_details(customer_response_2, conversation_history)
    print(f"\nExtracted order details: {json.dumps(order_details, indent=2)}")
    
    # Save order
    order_id = save_order(order_details, customer_phone)
    print(f"Order saved with ID: {order_id}")
    
    # Save conversation
    new_conversation = Conversation(
        call_sid=call_sid,
        customer_phone=customer_phone,
        conversation_log=json.dumps(conversation_history),
        order_id=order_id
    )
    
    session.add(new_conversation)
    session.commit()
    print(f"Conversation saved with ID: {new_conversation.id}")
    
    # Final confirmation and goodbye
    customer_response_3 = "That sounds good. Thank you."
    print(f"\nCustomer: {customer_response_3}")
    
    # Update conversation history
    conversation_history.append({"customer": customer_response_3})
    
    # Generate final response
    ai_response_3 = generate_response(customer_response_3, conversation_history, {"order_id": order_id})
    print(f"Assistant: {ai_response_3}")
    
    print("\n=== Call Simulation Complete ===")

# Uncomment to run the simulation
simulate_call()

=== Simulating Call Flow ===
Assistant: Thank you for calling Mario's Italian Restaurant. This is Mario's virtual assistant. How can I help you today?
Customer: Hi, I'd like to place an order for delivery.
Intent classified as: new_order
Assistant: Of course! May I have your name, please? Also, could you let me know what you'd like to order?
Customer: I'd like a Margherita pizza and a tiramisu for dessert. My name is John.
Assistant: Thank you, John. Just to confirm, you'd like one Margherita Pizza for $16 and one Tiramisu for $8. Could you please provide your delivery address and let me know if there's anything else you'd like to add to your order?

Extracted order details: {
  "customer_name": "John",
  "order_items": [
    {
      "item": "Margherita pizza",
      "quantity": 1,
      "special_instructions": null
    },
    {
      "item": "Tiramisu",
      "quantity": 1,
      "special_instructions": null
    }
  ],
  "is_delivery": true,
  "address": null,
  "reservation_time": nu

In [10]:
def measure_latency():
    """Measure latency of key operations"""
    import time
    
    print("=== Latency Testing ===")
    
    # Test intent classification latency
    start_time = time.time()
    classify_intent("I'd like to place an order for delivery")
    intent_latency = time.time() - start_time
    print(f"Intent classification latency: {intent_latency:.2f} seconds")
    
    # Test response generation latency
    conversation_history = [{"customer": "I'd like to place an order for delivery"}]
    start_time = time.time()
    generate_response("What's your most popular pizza?", conversation_history)
    response_latency = time.time() - start_time
    print(f"Response generation latency: {response_latency:.2f} seconds")
    
    # Test order parsing latency
    start_time = time.time()
    parse_order_details("I'd like a Margherita pizza and a tiramisu. My name is John.", conversation_history)
    parsing_latency = time.time() - start_time
    print(f"Order parsing latency: {parsing_latency:.2f} seconds")
    
    # Estimate overall response time
    avg_speech_recognition_time = 1.0  # Estimated time for Twilio to convert speech to text
    avg_tts_time = 1.0  # Estimated time for text-to-speech
    
    total_latency = intent_latency + response_latency + avg_speech_recognition_time + avg_tts_time
    print(f"Estimated total response latency: {total_latency:.2f} seconds")
    
    if total_latency > 5.0:
        print("WARNING: Latency may be too high for smooth conversation. Consider optimization.")
    else:
        print("Latency is within acceptable range for conversational AI.")

# Uncomment to run latency testing
measure_latency()

=== Latency Testing ===
Intent classification latency: 1.30 seconds
Response generation latency: 1.43 seconds
Order parsing latency: 5.33 seconds
Estimated total response latency: 4.73 seconds
Latency is within acceptable range for conversational AI.


In [11]:
# Clean up resources
session.close()
print("Database session closed.")

Database session closed.
